# Libraries

In [161]:
import numpy as np
import pandas as pd
import os, pathlib, glob, math
import warnings

from sklearn.metrics import mean_squared_error, mean_absolute_error,  r2_score, explained_variance_score

from plotly.subplots import make_subplots
import plotly.graph_objects as go

from keras.models import Sequential, load_model
from keras.layers import LSTM,Dense,Flatten,Conv1D,TimeDistributed,Reshape,ConvLSTM1D,Dropout
from keras.callbacks import ModelCheckpoint

# Data Preprocessing

In [2]:
data_dir = "PV"

In [3]:
data_dir = pathlib.Path(data_dir)
folder = list(data_dir.glob('*'))
print('Folder Structure:')
for f in folder:
    print(f)

Folder Structure:
PV\2_Febrero_SFV_63176617_2016.xlsx
PV\3_Marzo_SFV_63176617_2016.xlsx
PV\4_Abril_SFV_63176617_2016.xlsx
PV\5_Mayo_SFV_63176617_2016.xlsx
PV\6_Junio_SFV_63176617_2016.xlsx
PV\7_Julio_SFV_63176617_2016.xlsx


In [4]:
warnings.filterwarnings('ignore', category=UserWarning, module='openpyxl')
datosPV = pd.DataFrame()
for f in sorted(glob.glob(str(data_dir)+"/*.xlsx")):
  df = pd.read_excel(f, header=0)
  datosPV = pd.concat([datosPV,df], axis=0, sort=False, ignore_index=True)
datosPV = datosPV.dropna(axis=1)
datosPV = datosPV.drop(range(0, 960, 1),axis=0).reset_index(drop=True)
datosPV = datosPV.drop(labels=['MEDIDOR','KWHR','KVARHD','FECHA','INTERVALO'],axis=1)
rng = pd.date_range(start = '2016-02-11 00:00', end ='2016-07-31 23:45', freq='15min')
datosPV = datosPV.set_index(rng)
print(datosPV.shape)
datosPV.head(5)

(16512, 1)


,KWHD
2016-02-11 00:00:00,0.0
2016-02-11 00:15:00,0.0
2016-02-11 00:30:00,0.0
2016-02-11 00:45:00,0.0
2016-02-11 01:00:00,0.0


In [5]:
fin_train = '2016-06-10 09:45:00'
fin_validacion = '2016-07-14 19:15:00'

datosPV_train = datosPV.loc[: fin_train, :] 
datosPV_val = datosPV.loc[fin_train:fin_validacion, :] 
datosPV_test = datosPV.loc[fin_validacion:, :]

train_size = datosPV_train.shape[0]
val_size = datosPV_val.shape[0]
test_size = datosPV_test.shape[0]

print(f"Fechas train      : {datosPV_train.index.min()} --- {datosPV_train.index.max()}")
print(f"Fechas validacion : {datosPV_val.index.min()} --- {datosPV_val.index.max()}")
print(f"Fechas test       : {datosPV_test.index.min()} --- {datosPV_test.index.max()}")

Fechas train      : 2016-02-11 00:00:00 --- 2016-06-10 09:45:00
Fechas validacion : 2016-06-10 09:45:00 --- 2016-07-14 19:15:00
Fechas test       : 2016-07-14 19:15:00 --- 2016-07-31 23:45:00


In [6]:
def percentage_error(actual, predicted):
    res = np.empty(actual.shape)
    for j in range(actual.shape[0]):
        if actual[j] != 0:
            res[j] = (actual[j] - predicted[j]) / actual[j]
        else:
            res[j] = predicted[j] / np.mean(actual)
    return res

def MAPE(y_true, y_pred): 
    return np.mean(np.abs(percentage_error(np.asarray(y_true), np.asarray(y_pred))))*(100)

## Data conversion to forecasting windows

In [7]:
def convert_dataTS(train,val,test,lag,window):
    train = np.array(train)
    val = np.array(val)
    test = np.array(test)
    X_train, y_train = [], []
    X_val, y_val = [], []
    X_test, y_test = [], []

    for i in range(lag, len(train)-lag):
        X_train.append(train[i-lag:i])
        y_train.append(train[i:i+window])

    for i in range(lag, len(val)-lag):
        X_val.append(val[i-lag:i])
        y_val.append(val[i:i+window])

    for i in range(lag, len(test)-lag):
        X_test.append(test[i-lag:i])
        y_test.append(test[i:i+window])
    
    X_train,X_val,X_test,y_train,y_val,y_test = np.array(X_train), np.array(X_val), np.array(X_test), np.array(y_train), np.array(y_val), np.array(y_test) 
    return X_train,X_val,X_test,y_train,y_val,y_test
    

In [126]:
X_train_15,X_val_15,X_test_15,y_train_15,y_val_15,y_test_15 =  convert_dataTS(datosPV_train.KWHD,datosPV_val.KWHD,datosPV_test.KWHD,96,1)
print(X_train_15.shape, X_val_15.shape, X_test_15.shape)
X_train_30,X_val_30,X_test_30,y_train_30,y_val_30,y_test_30 =  convert_dataTS(datosPV_train.KWHD,datosPV_val.KWHD,datosPV_test.KWHD,96,2)
print(X_train_30.shape, X_val_30.shape, X_test_30.shape) 
X_train_60,X_val_60,X_test_60,y_train_60,y_val_60,y_test_60 =  convert_dataTS(datosPV_train.KWHD,datosPV_val.KWHD,datosPV_test.KWHD,96,4)
print(X_train_60.shape, X_val_60.shape, X_test_60.shape) 

(11368, 96) (3111, 96) (1459, 96)
(11368, 96) (3111, 96) (1459, 96)
(11368, 96) (3111, 96) (1459, 96)


In [127]:
X_train_15l,X_val_15l,X_test_15l,y_train_15l,y_val_15l,y_test_15l =  convert_dataTS(datosPV_train,datosPV_val,datosPV_test,96,1)
print(X_train_15l.shape, X_val_15l.shape, X_test_15l.shape)
X_train_30l,X_val_30l,X_test_30l,y_train_30l,y_val_30l,y_test_30l =  convert_dataTS(datosPV_train,datosPV_val,datosPV_test,96,2)
print(X_train_30l.shape, X_val_30l.shape, X_test_30l.shape) 
X_train_60l,X_val_60l,X_test_60l,y_train_60l,y_val_60l,y_test_60l =  convert_dataTS(datosPV_train,datosPV_val,datosPV_test,96,4)
print(X_train_60l.shape, X_val_60l.shape, X_test_60l.shape) 

(11368, 96, 1) (3111, 96, 1) (1459, 96, 1)
(11368, 96, 1) (3111, 96, 1) (1459, 96, 1)
(11368, 96, 1) (3111, 96, 1) (1459, 96, 1)


# DL models

## Classic MLP

### 15 mins ahead

In [122]:
model_MLP_15 = Sequential()
model_MLP_15.add(Dense(96, activation = 'relu', input_dim=19))
model_MLP_15.add(Dense(96, activation = 'relu'))
model_MLP_15.add(Dense(96, activation = 'relu'))
model_MLP_15.add(Dense(1))
model_MLP_15.summary()

Model: "sequential_31"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_51 (Dense)             (None, 96)                1920      
_________________________________________________________________
dense_52 (Dense)             (None, 96)                9312      
_________________________________________________________________
dense_53 (Dense)             (None, 96)                9312      
_________________________________________________________________
dense_54 (Dense)             (None, 1)                 97        
Total params: 20,641
Trainable params: 20,641
Non-trainable params: 0
_________________________________________________________________


In [123]:
checkpoint_MLP_15 = ModelCheckpoint(filepath='modelMLP15.best.hdf5', verbose = 1, save_best_only=True)
model_MLP_15.compile(optimizer='adam', loss='mae')
model_MLP_15.fit(X_train_15, y_train_15, epochs = 100, validation_data=(X_val_15, y_val_15), callbacks=[checkpoint_MLP_15])

Epoch 1/100
361/361 [==============================] - 2s 4ms/step - loss: 0.0145 - val_loss: 0.0134

Epoch 00001: val_loss improved from inf to 0.01343, saving model to modelMLP15.best.hdf5
Epoch 2/100
361/361 [==============================] - 1s 3ms/step - loss: 0.0127 - val_loss: 0.0145

Epoch 00002: val_loss did not improve from 0.01343
Epoch 3/100
361/361 [==============================] - 1s 3ms/step - loss: 0.0121 - val_loss: 0.0120

Epoch 00003: val_loss improved from 0.01343 to 0.01202, saving model to modelMLP15.best.hdf5
Epoch 4/100
361/361 [==============================] - 1s 3ms/step - loss: 0.0118 - val_loss: 0.0120

Epoch 00004: val_loss improved from 0.01202 to 0.01200, saving model to modelMLP15.best.hdf5
Epoch 5/100
361/361 [==============================] - 1s 3ms/step - loss: 0.0118 - val_loss: 0.0119

Epoch 00005: val_loss improved from 0.01200 to 0.01192, saving model to modelMLP15.best.hdf5
Epoch 6/100
361/361 [==============================] - 1s 3ms/step - lo

In [128]:
model_MLP_15 = load_model('modelMLP15.best.hdf5')
predictionsMLP = model_MLP_15.predict(X_test_15)
MAPE_MLP = MAPE(y_test_15, predictionsMLP)
print("MSE:", mean_squared_error(y_test_15, predictionsMLP))
print("RMSE:", math.sqrt(mean_squared_error(y_test_15, predictionsMLP)))
print("MAE:",mean_absolute_error(y_test_15, predictionsMLP))
print("R2:", r2_score(y_test_15, predictionsMLP))
print("Explained Variance:", explained_variance_score(y_test_15, predictionsMLP))
print("Mean Absolute Percentage Error MLP:",MAPE_MLP)

MSE: 0.0006859858871857158
RMSE: 0.026191332291155327
MAE: 0.012657416017301895
R2: 0.9029129451274454
Explained Variance: 0.9029585466106664
Mean Absolute Percentage Error MLP: 14.068231324813016


### 30 min ahead

In [19]:
model_MLP_30 = Sequential()
model_MLP_30.add(Dense(96, activation = 'relu', input_dim=96))
model_MLP_30.add(Dense(96, activation = 'relu'))
model_MLP_30.add(Dense(96, activation = 'relu'))
model_MLP_30.add(Dense(2))
model_MLP_30.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_12 (Dense)             (None, 96)                9312      
_________________________________________________________________
dense_13 (Dense)             (None, 96)                9312      
_________________________________________________________________
dense_14 (Dense)             (None, 96)                9312      
_________________________________________________________________
dense_15 (Dense)             (None, 2)                 194       
Total params: 28,130
Trainable params: 28,130
Non-trainable params: 0
_________________________________________________________________


In [20]:
checkpoint_MLP_30 = ModelCheckpoint(filepath='modelMLP30.best.hdf5', verbose = 1, save_best_only=True)
model_MLP_30.compile(optimizer='adam', loss='mae')
model_MLP_30.fit(X_train_30, y_train_30, epochs = 100, validation_data=(X_val_30, y_val_30), callbacks=[checkpoint_MLP_30])

Epoch 1/100
356/356 [==============================] - 2s 5ms/step - loss: 0.0181 - val_loss: 0.0147

Epoch 00001: val_loss improved from inf to 0.01465, saving model to modelMLP30.best.hdf5
Epoch 2/100
356/356 [==============================] - 2s 5ms/step - loss: 0.0144 - val_loss: 0.0135

Epoch 00002: val_loss improved from 0.01465 to 0.01349, saving model to modelMLP30.best.hdf5
Epoch 3/100
356/356 [==============================] - 2s 4ms/step - loss: 0.0137 - val_loss: 0.0146

Epoch 00003: val_loss did not improve from 0.01349
Epoch 4/100
356/356 [==============================] - 2s 4ms/step - loss: 0.0136 - val_loss: 0.0134

Epoch 00004: val_loss improved from 0.01349 to 0.01340, saving model to modelMLP30.best.hdf5
Epoch 5/100
356/356 [==============================] - 2s 5ms/step - loss: 0.0134 - val_loss: 0.0142

Epoch 00005: val_loss did not improve from 0.01340
Epoch 6/100
356/356 [==============================] - 2s 4ms/step - loss: 0.0133 - val_loss: 0.0138

Epoch 00006

In [21]:
model_MLP_30 = load_model('modelMLP30.best.hdf5')
predictionsMLP2 = model_MLP_30.predict(X_test_30)
MAPE_MLP1 = MAPE(y_test_30[:,0], predictionsMLP2[:,0])
MAPE_MLP2 = MAPE(y_test_30[:,1], predictionsMLP2[:,1])
print("MSE:", mean_squared_error(y_test_30, predictionsMLP2))
print("RMSE:", math.sqrt(mean_squared_error(y_test_30, predictionsMLP2)))
print("MAE:",mean_absolute_error(y_test_30, predictionsMLP2))
print("R2:", r2_score(y_test_30, predictionsMLP2))
print("Explained Variance:", explained_variance_score(y_test_30, predictionsMLP2))
print("Mean Absolute Percentage Error MLP:",(MAPE_MLP1+MAPE_MLP2)/2)

MSE: 0.0007768263181243014
RMSE: 0.02787160415412614
MAE: 0.013796466642439833
R2: 0.8900563688218248
Explained Variance: 0.8902781767692274
Mean Absolute Percentage Error MLP: 16.304058845022023


### 1 hour ahead

In [24]:
model_MLP_60 = Sequential()
model_MLP_60.add(Dense(96, activation = 'relu', input_dim=96))
model_MLP_60.add(Dense(96, activation = 'relu'))
model_MLP_60.add(Dense(96, activation = 'relu'))
model_MLP_60.add(Dense(4))
model_MLP_60.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_16 (Dense)             (None, 96)                9312      
_________________________________________________________________
dense_17 (Dense)             (None, 96)                9312      
_________________________________________________________________
dense_18 (Dense)             (None, 96)                9312      
_________________________________________________________________
dense_19 (Dense)             (None, 4)                 388       
Total params: 28,324
Trainable params: 28,324
Non-trainable params: 0
_________________________________________________________________


In [25]:
checkpoint_MLP_60 = ModelCheckpoint(filepath='modelMLP60.best.hdf5', verbose = 1, save_best_only=True)
model_MLP_60.compile(optimizer='adam', loss='mae')
model_MLP_60.fit(X_train_60, y_train_60, epochs = 100, validation_data=(X_val_60, y_val_60), callbacks=checkpoint_MLP_60)

Epoch 1/100
356/356 [==============================] - 2s 5ms/step - loss: 0.0193 - val_loss: 0.0155

Epoch 00001: val_loss improved from inf to 0.01550, saving model to modelMLP60.best.hdf5
Epoch 2/100
356/356 [==============================] - 2s 5ms/step - loss: 0.0159 - val_loss: 0.0145

Epoch 00002: val_loss improved from 0.01550 to 0.01446, saving model to modelMLP60.best.hdf5
Epoch 3/100
356/356 [==============================] - 2s 5ms/step - loss: 0.0151 - val_loss: 0.0142

Epoch 00003: val_loss improved from 0.01446 to 0.01419, saving model to modelMLP60.best.hdf5
Epoch 4/100
356/356 [==============================] - 2s 5ms/step - loss: 0.0148 - val_loss: 0.0159

Epoch 00004: val_loss did not improve from 0.01419
Epoch 5/100
356/356 [==============================] - 2s 5ms/step - loss: 0.0148 - val_loss: 0.0147

Epoch 00005: val_loss did not improve from 0.01419
Epoch 6/100
356/356 [==============================] - 2s 4ms/step - loss: 0.0145 - val_loss: 0.0148

Epoch 00006

In [26]:
model_MLP_60 = load_model('modelMLP60.best.hdf5')
predictionsMLP3 = model_MLP_60.predict(X_test_60)
MAPE_MLP1 = MAPE(y_test_60[:,0], predictionsMLP3[:,0])
MAPE_MLP2 = MAPE(y_test_60[:,1], predictionsMLP3[:,1])
MAPE_MLP3 = MAPE(y_test_60[:,2], predictionsMLP3[:,2])
MAPE_MLP4 = MAPE(y_test_60[:,3], predictionsMLP3[:,3])
print("MSE:", mean_squared_error(y_test_60, predictionsMLP3))
print("RMSE:", math.sqrt(mean_squared_error(y_test_60, predictionsMLP3)))
print("MAE:",mean_absolute_error(y_test_60, predictionsMLP3))
print("R2:", r2_score(y_test_60, predictionsMLP3))
print("Explained Variance:", explained_variance_score(y_test_60, predictionsMLP3))
print("Mean Absolute Percentage Error MLP:",(MAPE_MLP1+MAPE_MLP2+MAPE_MLP3+MAPE_MLP4)/4)

MSE: 0.0009249025842046955
RMSE: 0.03041221110351392
MAE: 0.015289164115092159
R2: 0.8690992488011058
Explained Variance: 0.8693535716882763
Mean Absolute Percentage Error MLP: 17.849770009094513


## MLP w/ReLU

### 15 mins ahead

In [27]:
predictionsMLPs = predictionsMLP * (predictionsMLP > 0)
MAPE1 = MAPE(y_test_15, predictionsMLPs)
print("MSE:", mean_squared_error(y_test_15, predictionsMLPs))
print("RMSE:", math.sqrt(mean_squared_error(y_test_15, predictionsMLPs)))
print("MAE:",mean_absolute_error(y_test_15, predictionsMLPs))
print("R2:", r2_score(y_test_15, predictionsMLPs))
print("Explained Variance:", explained_variance_score(y_test_15, predictionsMLPs))
print("Mean Absolute Percentage Error MLP:",MAPE1)

MSE: 0.0006859854350890066
RMSE: 0.026191323660498842
MAE: 0.012646803458305995
R2: 0.9029130091123447
Explained Variance: 0.9029603316858134
Mean Absolute Percentage Error MLP: 14.045744367412134


### 30 min ahead

In [28]:
predictionsMLPs2 = predictionsMLP2 * (predictionsMLP2 > 0)
MAPE1 = MAPE(y_test_30[:,0], predictionsMLPs2[:,0])
MAPE2 = MAPE(y_test_30[:,1], predictionsMLPs2[:,1])
print("MSE:", mean_squared_error(y_test_30, predictionsMLPs2))
print("RMSE:", math.sqrt(mean_squared_error(y_test_30, predictionsMLPs2)))
print("MAE:",mean_absolute_error(y_test_30, predictionsMLPs2))
print("R2:", r2_score(y_test_30, predictionsMLPs2))
print("Explained Variance:", explained_variance_score(y_test_30, predictionsMLPs2))
print("Mean Absolute Percentage Error MLP:",(MAPE1+MAPE2)/2)

MSE: 0.0007768053644317282
RMSE: 0.02787122825481016
MAE: 0.013712497178909409
R2: 0.8900593343818146
Explained Variance: 0.8902535940890437
Mean Absolute Percentage Error MLP: 16.16341406642686


### 1 hour ahead

In [29]:
predictionsMLPs3 = predictionsMLP3 * (predictionsMLP3 > 0)
MAPE1 = MAPE(y_test_60[:,0], predictionsMLPs3[:,0])
MAPE2 = MAPE(y_test_60[:,1], predictionsMLPs3[:,1])
MAPE3 = MAPE(y_test_60[:,2], predictionsMLPs3[:,2])
MAPE4 = MAPE(y_test_60[:,3], predictionsMLPs3[:,3])
print("MSE:", mean_squared_error(y_test_60, predictionsMLPs3))
print("RMSE:", math.sqrt(mean_squared_error(y_test_60, predictionsMLPs3)))
print("MAE:",mean_absolute_error(y_test_60, predictionsMLPs3))
print("R2:", r2_score(y_test_60, predictionsMLPs3))
print("Explained Variance:", explained_variance_score(y_test_60, predictionsMLPs3))
print("Mean Absolute Percentage Error MLP:",(MAPE1+MAPE2+MAPE3+MAPE4)/4)

MSE: 0.0009248993058837457
RMSE: 0.030412157205363544
MAE: 0.015271479702527992
R2: 0.8690997127793477
Explained Variance: 0.8693581800910625
Mean Absolute Percentage Error MLP: 17.81633848249319


## Vanilla LSTM

### 15 min ahead

In [138]:
model_VLSTM_15 = Sequential()
model_VLSTM_15.add(LSTM(96, activation='tanh', input_shape=(96, 1)))
model_VLSTM_15.add(Flatten())
model_VLSTM_15.add(Dense(1))
model_VLSTM_15.summary()

Model: "sequential_33"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_14 (LSTM)               (None, 96)                37632     
_________________________________________________________________
flatten_16 (Flatten)         (None, 96)                0         
_________________________________________________________________
dense_56 (Dense)             (None, 1)                 97        
Total params: 37,729
Trainable params: 37,729
Non-trainable params: 0
_________________________________________________________________


In [139]:
checkpoint_VLSTM_15 = ModelCheckpoint(filepath='modelVLSTM15.best.hdf5', verbose = 1, save_best_only=True)
model_VLSTM_15.compile(optimizer='adam', loss='mae')
model_VLSTM_15.fit(X_train_15l, y_train_15l, epochs = 100, validation_data=(X_val_15l, y_val_15l), callbacks=[checkpoint_VLSTM_15])

Epoch 1/100
356/356 [==============================] - 8s 14ms/step - loss: 0.0194 - val_loss: 0.0171

Epoch 00001: val_loss improved from inf to 0.01708, saving model to modelVLSTM15.best.hdf5
Epoch 2/100
356/356 [==============================] - 4s 11ms/step - loss: 0.0140 - val_loss: 0.0129

Epoch 00002: val_loss improved from 0.01708 to 0.01295, saving model to modelVLSTM15.best.hdf5
Epoch 3/100
356/356 [==============================] - 4s 10ms/step - loss: 0.0124 - val_loss: 0.0124

Epoch 00003: val_loss improved from 0.01295 to 0.01238, saving model to modelVLSTM15.best.hdf5
Epoch 4/100
356/356 [==============================] - 4s 10ms/step - loss: 0.0121 - val_loss: 0.0124

Epoch 00004: val_loss did not improve from 0.01238
Epoch 5/100
356/356 [==============================] - 4s 11ms/step - loss: 0.0119 - val_loss: 0.0131

Epoch 00005: val_loss did not improve from 0.01238
Epoch 6/100
356/356 [==============================] - 4s 10ms/step - loss: 0.0119 - val_loss: 0.0122


In [179]:
model_VLSTM_15 = load_model('modelVLSTM15.best.hdf5')
predictionsVLSTM = model_VLSTM_15.predict(X_test_15l)
predictionsVLSTM = predictionsVLSTM * (predictionsVLSTM > 0)
MAPE1 = MAPE(y_test_15[:,0], predictionsVLSTM[:,0])
print("MSE:", mean_squared_error(y_test_15, predictionsVLSTM))
print("RMSE:", math.sqrt(mean_squared_error(y_test_15, predictionsVLSTM)))
print("MAE:",mean_absolute_error(y_test_15, predictionsVLSTM))
print("R2:", r2_score(y_test_15, predictionsVLSTM))
print("Explained Variance:", explained_variance_score(y_test_15, predictionsVLSTM))
print("Mean Absolute Percentage Error MLP:",MAPE1)

MSE: 0.0006710860085170262
RMSE: 0.02590532780176746
MAE: 0.012168305751198625
R2: 0.9050217134926899
Explained Variance: 0.9053449823442833
Mean Absolute Percentage Error MLP: 14.57795782934372


### 30 min ahead

In [143]:
model_VLSTM_30 = Sequential()
model_VLSTM_30.add(LSTM(96, activation='tanh', input_shape=(96, 1)))
model_VLSTM_30.add(Flatten())
model_VLSTM_30.add(Dense(2))
model_VLSTM_30.summary()

Model: "sequential_34"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_15 (LSTM)               (None, 96)                37632     
_________________________________________________________________
flatten_17 (Flatten)         (None, 96)                0         
_________________________________________________________________
dense_57 (Dense)             (None, 2)                 194       
Total params: 37,826
Trainable params: 37,826
Non-trainable params: 0
_________________________________________________________________


In [144]:
checkpoint_VLSTM_30 = ModelCheckpoint(filepath='modelVLSTM30.best.hdf5', verbose = 1, save_best_only=True)
model_VLSTM_30.compile(optimizer='adam', loss='mae')
model_VLSTM_30.fit(X_train_30l, y_train_30l, epochs = 100, validation_data=(X_val_30l, y_val_30l), callbacks=checkpoint_VLSTM_30)

Epoch 1/100
356/356 [==============================] - 7s 13ms/step - loss: 0.0216 - val_loss: 0.0168

Epoch 00001: val_loss improved from inf to 0.01676, saving model to modelVLSTM30.best.hdf5
Epoch 2/100
356/356 [==============================] - 4s 10ms/step - loss: 0.0158 - val_loss: 0.0149

Epoch 00002: val_loss improved from 0.01676 to 0.01494, saving model to modelVLSTM30.best.hdf5
Epoch 3/100
356/356 [==============================] - 3s 10ms/step - loss: 0.0147 - val_loss: 0.0147

Epoch 00003: val_loss improved from 0.01494 to 0.01473, saving model to modelVLSTM30.best.hdf5
Epoch 4/100
356/356 [==============================] - 4s 10ms/step - loss: 0.0142 - val_loss: 0.0142

Epoch 00004: val_loss improved from 0.01473 to 0.01416, saving model to modelVLSTM30.best.hdf5
Epoch 5/100
356/356 [==============================] - 4s 10ms/step - loss: 0.0140 - val_loss: 0.0139

Epoch 00005: val_loss improved from 0.01416 to 0.01391, saving model to modelVLSTM30.best.hdf5
Epoch 6/100
35

In [180]:
model_VLSTM_30 = load_model('modelVLSTM30.best.hdf5')
predictionsVLSTM2 = model_VLSTM_30.predict(X_test_30l)
predictionsVLSTM2 = predictionsVLSTM2 * (predictionsVLSTM2 > 0)
MAPE1 = MAPE(y_test_30l[:,0], predictionsVLSTM2[:,0])
MAPE2 = MAPE(y_test_30l[:,1], predictionsVLSTM2[:,1])
print("MSE:", mean_squared_error(y_test_30, predictionsVLSTM2))
print("RMSE:", math.sqrt(mean_squared_error(y_test_30, predictionsVLSTM2)))
print("MAE:",mean_absolute_error(y_test_30, predictionsVLSTM2))
print("R2:", r2_score(y_test_30, predictionsVLSTM2))
print("Explained Variance:", explained_variance_score(y_test_30, predictionsVLSTM2))
print("Mean Absolute Percentage Error MLP:",(MAPE1+MAPE2)/2)

MSE: 0.0008134290596948779
RMSE: 0.02852067775658352
MAE: 0.013737960533233201
R2: 0.8848760109664648
Explained Variance: 0.8852308939712108
Mean Absolute Percentage Error MLP: 16.546445148897632


### 1 hour ahead

In [148]:
model_VLSTM_60 = Sequential()
model_VLSTM_60.add(LSTM(96, activation='tanh', input_shape=(96, 1)))
model_VLSTM_60.add(Flatten())
model_VLSTM_60.add(Dense(4))
model_VLSTM_60.summary()

Model: "sequential_35"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_16 (LSTM)               (None, 96)                37632     
_________________________________________________________________
flatten_18 (Flatten)         (None, 96)                0         
_________________________________________________________________
dense_58 (Dense)             (None, 4)                 388       
Total params: 38,020
Trainable params: 38,020
Non-trainable params: 0
_________________________________________________________________


In [149]:
checkpoint_VLSTM_60 = ModelCheckpoint(filepath='modelVLSTM60.best.hdf5', verbose = 1, save_best_only=True)
model_VLSTM_60.compile(optimizer='adam', loss='mae')
model_VLSTM_60.fit(X_train_60l, y_train_60l, epochs = 100, validation_data=(X_val_60l, y_val_60l), callbacks=checkpoint_VLSTM_60)

Epoch 1/100
356/356 [==============================] - 6s 12ms/step - loss: 0.0245 - val_loss: 0.0191

Epoch 00001: val_loss improved from inf to 0.01913, saving model to modelVLSTM60.best.hdf5
Epoch 2/100
356/356 [==============================] - 3s 10ms/step - loss: 0.0186 - val_loss: 0.0175

Epoch 00002: val_loss improved from 0.01913 to 0.01750, saving model to modelVLSTM60.best.hdf5
Epoch 3/100
356/356 [==============================] - 3s 10ms/step - loss: 0.0174 - val_loss: 0.0169

Epoch 00003: val_loss improved from 0.01750 to 0.01690, saving model to modelVLSTM60.best.hdf5
Epoch 4/100
356/356 [==============================] - 4s 10ms/step - loss: 0.0163 - val_loss: 0.0182

Epoch 00004: val_loss did not improve from 0.01690
Epoch 5/100
356/356 [==============================] - 4s 10ms/step - loss: 0.0159 - val_loss: 0.0156

Epoch 00005: val_loss improved from 0.01690 to 0.01559, saving model to modelVLSTM60.best.hdf5
Epoch 6/100
356/356 [==============================] - 4s 

In [181]:
model_VLSTM_60 = load_model('modelVLSTM60.best.hdf5')
predictionsVLSTM3 = model_VLSTM_60.predict(X_test_60l)
predictionsVLSTM3 = predictionsVLSTM3 * (predictionsVLSTM3 > 0)
MAPE1 = MAPE(y_test_60[:,0], predictionsVLSTM3[:,0])
MAPE2 = MAPE(y_test_60[:,1], predictionsVLSTM3[:,1])
MAPE3 = MAPE(y_test_60[:,2], predictionsVLSTM3[:,2])
MAPE4 = MAPE(y_test_60[:,3], predictionsVLSTM3[:,3])
print("MSE:", mean_squared_error(y_test_60, predictionsVLSTM3))
print("RMSE:", math.sqrt(mean_squared_error(y_test_60, predictionsVLSTM3)))
print("MAE:",mean_absolute_error(y_test_60, predictionsVLSTM3))
print("R2:", r2_score(y_test_60, predictionsVLSTM3))
print("Explained Variance:", explained_variance_score(y_test_60, predictionsVLSTM3))
print("Mean Absolute Percentage Error MLP:",(MAPE1+MAPE2+MAPE3+MAPE4)/4)

MSE: 0.00092362781662001
RMSE: 0.03039124572339887
MAE: 0.01519131617647595
R2: 0.8692796656766655
Explained Variance: 0.8693047246651535
Mean Absolute Percentage Error MLP: 18.542279662100583


## LSTM

### 15 min ahead

In [152]:
model_LSTM_15 = Sequential()
model_LSTM_15.add(LSTM(96, activation='tanh', return_sequences=True, input_shape=(96, 1)))
model_LSTM_15.add(LSTM(96, activation='tanh', return_sequences=True))
model_LSTM_15.add(LSTM(96, activation='tanh'))
model_LSTM_15.add(Flatten())
model_LSTM_15.add(Dense(1))
model_LSTM_15.summary()

Model: "sequential_36"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_17 (LSTM)               (None, 96, 96)            37632     
_________________________________________________________________
lstm_18 (LSTM)               (None, 96, 96)            74112     
_________________________________________________________________
lstm_19 (LSTM)               (None, 96)                74112     
_________________________________________________________________
flatten_19 (Flatten)         (None, 96)                0         
_________________________________________________________________
dense_59 (Dense)             (None, 1)                 97        
Total params: 185,953
Trainable params: 185,953
Non-trainable params: 0
_________________________________________________________________


In [153]:
checkpoint_LSTM_15 = ModelCheckpoint(filepath='modelLSTM15.best.hdf5', verbose = 1, save_best_only=True)
model_LSTM_15.compile(optimizer='adam', loss='mae')
model_LSTM_15.fit(X_train_15l, y_train_15l, epochs = 100, validation_data=(X_val_15l, y_val_15l), callbacks=checkpoint_LSTM_15)

Epoch 1/100
356/356 [==============================] - 17s 33ms/step - loss: 0.0209 - val_loss: 0.0147

Epoch 00001: val_loss improved from inf to 0.01475, saving model to modelLSTM15.best.hdf5
Epoch 2/100
356/356 [==============================] - 9s 26ms/step - loss: 0.0136 - val_loss: 0.0133

Epoch 00002: val_loss improved from 0.01475 to 0.01329, saving model to modelLSTM15.best.hdf5
Epoch 3/100
356/356 [==============================] - 9s 25ms/step - loss: 0.0129 - val_loss: 0.0148

Epoch 00003: val_loss did not improve from 0.01329
Epoch 4/100
356/356 [==============================] - 9s 25ms/step - loss: 0.0121 - val_loss: 0.0117

Epoch 00004: val_loss improved from 0.01329 to 0.01169, saving model to modelLSTM15.best.hdf5
Epoch 5/100
356/356 [==============================] - 9s 25ms/step - loss: 0.0118 - val_loss: 0.0126

Epoch 00005: val_loss did not improve from 0.01169
Epoch 6/100
356/356 [==============================] - 9s 25ms/step - loss: 0.0121 - val_loss: 0.0123

E

In [182]:
model_LSTM_15 = load_model('modelLSTM15.best.hdf5')
predictionsLSTM = model_LSTM_15.predict(X_test_15l)
predictionsLSTM = predictionsLSTM * (predictionsLSTM > 0)
MAPE1 = MAPE(y_test_15[:,0], predictionsLSTM[:,0])
print("MSE:", mean_squared_error(y_test_15, predictionsLSTM))
print("RMSE:", math.sqrt(mean_squared_error(y_test_15, predictionsLSTM)))
print("MAE:",mean_absolute_error(y_test_15, predictionsLSTM))
print("R2:", r2_score(y_test_15, predictionsLSTM))
print("Explained Variance:", explained_variance_score(y_test_15, predictionsLSTM))
print("Mean Absolute Percentage Error MLP:",MAPE1)

MSE: 0.0006790416289701103
RMSE: 0.026058427215972
MAE: 0.012260419320798977
R2: 0.9038957606503617
Explained Variance: 0.9040924146205538
Mean Absolute Percentage Error MLP: 14.019197990083414


### 30 min ahead

In [155]:
model_LSTM_30 = Sequential()
model_LSTM_30.add(LSTM(96, activation='tanh', return_sequences=True, input_shape=(96, 1)))
model_LSTM_30.add(LSTM(96, activation='tanh', return_sequences=True))
model_LSTM_30.add(LSTM(96, activation='tanh'))
model_LSTM_30.add(Flatten())
model_LSTM_30.add(Dense(2))
model_LSTM_30.summary()

Model: "sequential_37"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_20 (LSTM)               (None, 96, 96)            37632     
_________________________________________________________________
lstm_21 (LSTM)               (None, 96, 96)            74112     
_________________________________________________________________
lstm_22 (LSTM)               (None, 96)                74112     
_________________________________________________________________
flatten_20 (Flatten)         (None, 96)                0         
_________________________________________________________________
dense_60 (Dense)             (None, 2)                 194       
Total params: 186,050
Trainable params: 186,050
Non-trainable params: 0
_________________________________________________________________


In [156]:
checkpoint_LSTM_30 = ModelCheckpoint(filepath='modelLSTM30.best.hdf5', verbose = 1, save_best_only=True)
model_LSTM_30.compile(optimizer='adam', loss='mae')
model_LSTM_30.fit(X_train_30l, y_train_30l, epochs = 100, validation_data=(X_val_30l, y_val_30l), callbacks=[checkpoint_LSTM_30])

Epoch 1/100
356/356 [==============================] - 14s 28ms/step - loss: 0.0234 - val_loss: 0.0164

Epoch 00001: val_loss improved from inf to 0.01640, saving model to modelLSTM30.best.hdf5
Epoch 2/100
356/356 [==============================] - 9s 25ms/step - loss: 0.0155 - val_loss: 0.0156

Epoch 00002: val_loss improved from 0.01640 to 0.01562, saving model to modelLSTM30.best.hdf5
Epoch 3/100
356/356 [==============================] - 9s 24ms/step - loss: 0.0145 - val_loss: 0.0144

Epoch 00003: val_loss improved from 0.01562 to 0.01439, saving model to modelLSTM30.best.hdf5
Epoch 4/100
356/356 [==============================] - 9s 25ms/step - loss: 0.0141 - val_loss: 0.0159

Epoch 00004: val_loss did not improve from 0.01439
Epoch 5/100
356/356 [==============================] - 9s 25ms/step - loss: 0.0142 - val_loss: 0.0146

Epoch 00005: val_loss did not improve from 0.01439
Epoch 6/100
356/356 [==============================] - 9s 25ms/step - loss: 0.0138 - val_loss: 0.0152

E

In [183]:
model_LSTM_30 = load_model('modelLSTM30.best.hdf5')
predictionsLSTM2 = model_LSTM_30.predict(X_test_30l)
predictionsLSTM2 = predictionsLSTM2 * (predictionsLSTM2 > 0)
MAPE1 = MAPE(y_test_30l[:,0], predictionsLSTM2[:,0])
MAPE2 = MAPE(y_test_30l[:,1], predictionsLSTM2[:,1])
print("MSE:", mean_squared_error(y_test_30, predictionsLSTM2))
print("RMSE:", math.sqrt(mean_squared_error(y_test_30, predictionsLSTM2)))
print("MAE:",mean_absolute_error(y_test_30, predictionsLSTM2))
print("R2:", r2_score(y_test_30, predictionsLSTM2))
print("Explained Variance:", explained_variance_score(y_test_30, predictionsLSTM2))
print("Mean Absolute Percentage Error MLP:",(MAPE1+MAPE2)/2)

MSE: 0.0008208494332684572
RMSE: 0.028650470035733396
MAE: 0.013856538949480031
R2: 0.8838258112032182
Explained Variance: 0.8843241900892453
Mean Absolute Percentage Error MLP: 16.447732634653985


### 1 hour ahead

In [158]:
model_LSTM_60 = Sequential()
model_LSTM_60.add(LSTM(96, activation='tanh', return_sequences=True, input_shape=(96, 1)))
model_LSTM_60.add(LSTM(96, activation='tanh', return_sequences=True))
model_LSTM_60.add(LSTM(96, activation='tanh'))
model_LSTM_60.add(Flatten())
model_LSTM_60.add(Dense(4))
model_LSTM_60.summary()

Model: "sequential_38"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_23 (LSTM)               (None, 96, 96)            37632     
_________________________________________________________________
lstm_24 (LSTM)               (None, 96, 96)            74112     
_________________________________________________________________
lstm_25 (LSTM)               (None, 96)                74112     
_________________________________________________________________
flatten_21 (Flatten)         (None, 96)                0         
_________________________________________________________________
dense_61 (Dense)             (None, 4)                 388       
Total params: 186,244
Trainable params: 186,244
Non-trainable params: 0
_________________________________________________________________


In [159]:
checkpoint_LSTM_60 = ModelCheckpoint(filepath='modelLSTM60.best.hdf5', verbose = 1, save_best_only=True)
model_LSTM_60.compile(optimizer='adam', loss='mae')
model_LSTM_60.fit(X_train_60l, y_train_60l, epochs = 100, validation_data=(X_val_60l, y_val_60l), callbacks=[checkpoint_LSTM_60])

Epoch 1/100
356/356 [==============================] - 14s 28ms/step - loss: 0.0261 - val_loss: 0.0198

Epoch 00001: val_loss improved from inf to 0.01975, saving model to modelLSTM60.best.hdf5
Epoch 2/100
356/356 [==============================] - 9s 24ms/step - loss: 0.0186 - val_loss: 0.0173

Epoch 00002: val_loss improved from 0.01975 to 0.01729, saving model to modelLSTM60.best.hdf5
Epoch 3/100
356/356 [==============================] - 9s 25ms/step - loss: 0.0169 - val_loss: 0.0179

Epoch 00003: val_loss did not improve from 0.01729
Epoch 4/100
356/356 [==============================] - 9s 24ms/step - loss: 0.0165 - val_loss: 0.0159

Epoch 00004: val_loss improved from 0.01729 to 0.01591, saving model to modelLSTM60.best.hdf5
Epoch 5/100
356/356 [==============================] - 9s 24ms/step - loss: 0.0159 - val_loss: 0.0154

Epoch 00005: val_loss improved from 0.01591 to 0.01539, saving model to modelLSTM60.best.hdf5
Epoch 6/100
356/356 [==============================] - 9s 24m

In [184]:
model_LSTM_60 = load_model('modelLSTM60.best.hdf5')
predictionsLSTM3 = model_LSTM_60.predict(X_test_60l)
predictionsLSTM3 = predictionsLSTM3 * (predictionsLSTM3 > 0)
MAPE1 = MAPE(y_test_60l[:,0], predictionsLSTM3[:,0])
MAPE2 = MAPE(y_test_60l[:,1], predictionsLSTM3[:,1])
print("MSE:", mean_squared_error(y_test_60, predictionsLSTM3))
print("RMSE:", math.sqrt(mean_squared_error(y_test_60, predictionsLSTM3)))
print("MAE:",mean_absolute_error(y_test_60, predictionsLSTM3))
print("R2:", r2_score(y_test_60, predictionsLSTM3))
print("Explained Variance:", explained_variance_score(y_test_60, predictionsLSTM3))
print("Mean Absolute Percentage Error MLP:",(MAPE1+MAPE2)/2)

MSE: 0.0009885369279446763
RMSE: 0.03144100710767192
MAE: 0.01546117433479708
R2: 0.8600931290865904
Explained Variance: 0.8605257141604543
Mean Absolute Percentage Error MLP: 17.137958640800218


## Conv-LSTM

### 15 min ahead

In [167]:
model_CLSTM_15 = Sequential()
model_CLSTM_15.add(Reshape((1, 96, 1), input_shape=(96,1)))
model_CLSTM_15.add(ConvLSTM1D(filters=96, kernel_size=3, activation='tanh', input_shape=(1, 96, 1), return_sequences=True))
model_CLSTM_15.add(ConvLSTM1D(filters=96, kernel_size=3, activation='tanh', return_sequences=True))
model_CLSTM_15.add(ConvLSTM1D(filters=96, kernel_size=3, activation='tanh'))
model_CLSTM_15.add(Flatten())
model_CLSTM_15.add(Dense(1))
model_CLSTM_15.summary()

Model: "sequential_41"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
reshape_19 (Reshape)         (None, 1, 96, 1)          0         
_________________________________________________________________
conv_lst_m1d_52 (ConvLSTM1D) (None, 1, 92, 96)         186624    
_________________________________________________________________
conv_lst_m1d_53 (ConvLSTM1D) (None, 1, 88, 96)         369024    
_________________________________________________________________
conv_lst_m1d_54 (ConvLSTM1D) (None, 84, 96)            369024    
_________________________________________________________________
flatten_23 (Flatten)         (None, 8064)              0         
_________________________________________________________________
dense_64 (Dense)             (None, 1)                 8065      
Total params: 932,737
Trainable params: 932,737
Non-trainable params: 0
_______________________________________________

In [168]:
checkpoint_CLSTM_15 = ModelCheckpoint(filepath='modelCLSTM15.best.hdf5', verbose = 1, save_best_only=True)
model_CLSTM_15.compile(optimizer='adam', loss='mae')
model_CLSTM_15.fit(X_train_15l, y_train_15l, epochs = 100, validation_data=(X_val_15l, y_val_15l), callbacks=[checkpoint_CLSTM_15])

Epoch 1/100
356/356 [==============================] - 51s 63ms/step - loss: 0.0266 - val_loss: 0.0181

Epoch 00001: val_loss improved from inf to 0.01806, saving model to modelCLSTM15.best.hdf5
Epoch 2/100
356/356 [==============================] - 20s 56ms/step - loss: 0.0157 - val_loss: 0.0142

Epoch 00002: val_loss improved from 0.01806 to 0.01419, saving model to modelCLSTM15.best.hdf5
Epoch 3/100
356/356 [==============================] - 20s 56ms/step - loss: 0.0139 - val_loss: 0.0147

Epoch 00003: val_loss did not improve from 0.01419
Epoch 4/100
356/356 [==============================] - 20s 56ms/step - loss: 0.0135 - val_loss: 0.0136

Epoch 00004: val_loss improved from 0.01419 to 0.01362, saving model to modelCLSTM15.best.hdf5
Epoch 5/100
356/356 [==============================] - 20s 57ms/step - loss: 0.0135 - val_loss: 0.0140

Epoch 00005: val_loss did not improve from 0.01362
Epoch 6/100
356/356 [==============================] - 20s 56ms/step - loss: 0.0134 - val_loss: 0

In [185]:
model_CLSTM_15 = load_model('modelCLSTM15.best.hdf5')
predictionsCLSTM = model_CLSTM_15.predict(X_test_15l)
predictionsCLSTM = predictionsCLSTM * (predictionsCLSTM > 0)
MAPE1 = MAPE(y_test_15[:,0], predictionsCLSTM[:,0])
print("MSE:", mean_squared_error(y_test_15, predictionsCLSTM))
print("RMSE:", math.sqrt(mean_squared_error(y_test_15, predictionsCLSTM)))
print("MAE:",mean_absolute_error(y_test_15, predictionsCLSTM))
print("R2:", r2_score(y_test_15, predictionsCLSTM))
print("Explained Variance:", explained_variance_score(y_test_15, predictionsCLSTM))
print("Mean Absolute Percentage Error CLSTM:",MAPE1)

MSE: 0.0006959721598898841
RMSE: 0.02638128427294403
MAE: 0.012474435561360195
R2: 0.9014995956342372
Explained Variance: 0.9017757574377407
Mean Absolute Percentage Error CLSTM: 14.3859543554592


### 30 min ahead

In [111]:
model_CLSTM_30 = Sequential()
model_CLSTM_30.add(Reshape((1, 96, 1), input_shape=(96,1)))
model_CLSTM_30.add(ConvLSTM1D(filters=96, kernel_size=3, activation='tanh', input_shape=(1, 96, 1), return_sequences=True))
model_CLSTM_30.add(ConvLSTM1D(filters=96, kernel_size=3, activation='tanh', return_sequences=True))
model_CLSTM_30.add(ConvLSTM1D(filters=96, kernel_size=3, activation='tanh'))
model_CLSTM_30.add(Flatten())
model_CLSTM_30.add(Dense(2))
model_CLSTM_30.summary()

Model: "sequential_28"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
reshape_15 (Reshape)         (None, 1, 96, 1)          0         
_________________________________________________________________
conv_lst_m1d_40 (ConvLSTM1D) (None, 1, 94, 96)         112128    
_________________________________________________________________
conv_lst_m1d_41 (ConvLSTM1D) (None, 1, 92, 96)         221568    
_________________________________________________________________
conv_lst_m1d_42 (ConvLSTM1D) (None, 90, 96)            221568    
_________________________________________________________________
flatten_14 (Flatten)         (None, 8640)              0         
_________________________________________________________________
dense_42 (Dense)             (None, 2)                 17282     
Total params: 572,546
Trainable params: 572,546
Non-trainable params: 0
_______________________________________________

In [112]:
checkpoint_CLSTM_30 = ModelCheckpoint(filepath='modelCLSTM30.best.hdf5', verbose = 1, save_best_only=True)
model_CLSTM_30.compile(optimizer='adam', loss='mae')
model_CLSTM_30.fit(X_train_30l, y_train_30l, epochs = 100, validation_data=(X_val_30l, y_val_30l), callbacks=[checkpoint_CLSTM_30])

Epoch 1/100
356/356 [==============================] - 50s 52ms/step - loss: 0.0256 - val_loss: 0.0204

Epoch 00001: val_loss improved from inf to 0.02040, saving model to modelCLSTM30.best.hdf5
Epoch 2/100
356/356 [==============================] - 18s 49ms/step - loss: 0.0172 - val_loss: 0.0146

Epoch 00002: val_loss improved from 0.02040 to 0.01457, saving model to modelCLSTM30.best.hdf5
Epoch 3/100
356/356 [==============================] - 18s 49ms/step - loss: 0.0153 - val_loss: 0.0149

Epoch 00003: val_loss did not improve from 0.01457
Epoch 4/100
356/356 [==============================] - 18s 52ms/step - loss: 0.0151 - val_loss: 0.0158

Epoch 00004: val_loss did not improve from 0.01457
Epoch 5/100
356/356 [==============================] - 18s 51ms/step - loss: 0.0151 - val_loss: 0.0148

Epoch 00005: val_loss did not improve from 0.01457
Epoch 6/100
356/356 [==============================] - 18s 49ms/step - loss: 0.0148 - val_loss: 0.0159

Epoch 00006: val_loss did not improve

In [186]:
model_CLSTM_30 = load_model('modelCLSTM30.best.hdf5')
predictionsCLSTM2 = model_CLSTM_30.predict(X_test_30l)
predictionsCLSTM2 = predictionsCLSTM2 * (predictionsCLSTM2 > 0)
MAPE1 = MAPE(y_test_30l[:,0], predictionsCLSTM2[:,0])
MAPE2 = MAPE(y_test_30l[:,1], predictionsCLSTM2[:,1])
print("MSE:", mean_squared_error(y_test_30, predictionsCLSTM2))
print("RMSE:", math.sqrt(mean_squared_error(y_test_30, predictionsCLSTM2)))
print("MAE:",mean_absolute_error(y_test_30, predictionsCLSTM2))
print("R2:", r2_score(y_test_30, predictionsCLSTM2))
print("Explained Variance:", explained_variance_score(y_test_30, predictionsCLSTM2))
print("Mean Absolute Percentage Error CLSTM:",(MAPE1+MAPE2)/2)

MSE: 0.0008084084603019654
RMSE: 0.028432524690958513
MAE: 0.0139155525009086
R2: 0.8855865725361096
Explained Variance: 0.8859176481740505
Mean Absolute Percentage Error CLSTM: 19.107778965224284


### 1 hour ahead

In [134]:
model_CLSTM_60 = Sequential()
model_CLSTM_60.add(Reshape((1, 96, 1), input_shape=(96,1)))
model_CLSTM_60.add(ConvLSTM1D(filters=96, kernel_size=3, activation='tanh', input_shape=(1, 96, 1), return_sequences=True))
model_CLSTM_60.add(ConvLSTM1D(filters=128, kernel_size=3, activation='tanh', return_sequences=True))
model_CLSTM_60.add(ConvLSTM1D(filters=96, kernel_size=3, activation='tanh'))
model_CLSTM_60.add(Flatten())
model_CLSTM_60.add(Dense(4))
model_CLSTM_60.summary()

Model: "sequential_32"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
reshape_16 (Reshape)         (None, 1, 96, 1)          0         
_________________________________________________________________
conv_lst_m1d_43 (ConvLSTM1D) (None, 1, 94, 96)         112128    
_________________________________________________________________
conv_lst_m1d_44 (ConvLSTM1D) (None, 1, 92, 256)        1082368   
_________________________________________________________________
conv_lst_m1d_45 (ConvLSTM1D) (None, 90, 96)            405888    
_________________________________________________________________
flatten_15 (Flatten)         (None, 8640)              0         
_________________________________________________________________
dense_55 (Dense)             (None, 4)                 34564     
Total params: 1,634,948
Trainable params: 1,634,948
Non-trainable params: 0
___________________________________________

In [135]:
checkpoint_CLSTM_60 = ModelCheckpoint(filepath='modelCLSTM60.best.hdf5', verbose = 1, save_best_only=True)
model_CLSTM_60.compile(optimizer='adam', loss='mae')
model_CLSTM_60.fit(X_train_60l, y_train_60l, epochs = 100, validation_data=(X_val_60l, y_val_60l), callbacks=[checkpoint_CLSTM_60])

Epoch 1/100
356/356 [==============================] - 39s 76ms/step - loss: 0.0260 - val_loss: 0.0209

Epoch 00001: val_loss improved from inf to 0.02088, saving model to modelCLSTM60.best.hdf5
Epoch 2/100
356/356 [==============================] - 26s 73ms/step - loss: 0.0188 - val_loss: 0.0168

Epoch 00002: val_loss improved from 0.02088 to 0.01679, saving model to modelCLSTM60.best.hdf5
Epoch 3/100
356/356 [==============================] - 26s 73ms/step - loss: 0.0179 - val_loss: 0.0168

Epoch 00003: val_loss improved from 0.01679 to 0.01677, saving model to modelCLSTM60.best.hdf5
Epoch 4/100
356/356 [==============================] - 27s 76ms/step - loss: 0.0176 - val_loss: 0.0164

Epoch 00004: val_loss improved from 0.01677 to 0.01636, saving model to modelCLSTM60.best.hdf5
Epoch 5/100
356/356 [==============================] - 25s 71ms/step - loss: 0.0169 - val_loss: 0.0163

Epoch 00005: val_loss improved from 0.01636 to 0.01627, saving model to modelCLSTM60.best.hdf5
Epoch 6/1

In [187]:
model_CLSTM_60 = load_model('modelCLSTM60.best.hdf5')
predictionsCLSTM3 = model_CLSTM_60.predict(X_test_60l)
predictionsCLSTM3 = predictionsCLSTM3 * (predictionsCLSTM3 > 0)
MAPE1 = MAPE(y_test_60l[:,0], predictionsCLSTM3[:,0])
MAPE2 = MAPE(y_test_60l[:,1], predictionsCLSTM3[:,1])
print("MSE:", mean_squared_error(y_test_60, predictionsCLSTM3))
print("RMSE:", math.sqrt(mean_squared_error(y_test_60, predictionsCLSTM3)))
print("MAE:",mean_absolute_error(y_test_60, predictionsCLSTM3))
print("R2:", r2_score(y_test_60, predictionsCLSTM3))
print("Explained Variance:", explained_variance_score(y_test_60, predictionsCLSTM3))
print("Mean Absolute Percentage Error CLSTM:",(MAPE1+MAPE2)/2)

MSE: 0.0009757337589393553
RMSE: 0.031236737328654465
MAE: 0.01575021125379289
R2: 0.8619051517462136
Explained Variance: 0.8626627921963314
Mean Absolute Percentage Error CLSTM: 19.400046421870325


# Comparison

In [188]:
figPVCom2 = make_subplots(rows=3, cols=1, vertical_spacing = 0.11, subplot_titles=['(a)','(b)','(c)'])


figPVCom2.append_trace(go.Scatter(y=datosPV_test.KWHD[95:,], x=datosPV_test.index,
                                 mode='lines', line=dict(width=2, color="blue"),
                                 name="Test data"), row=1, col=1)
figPVCom2.append_trace(go.Scatter(y=predictionsMLP[:,0], x=datosPV_test.index,
                                 mode='lines', line=dict(width=2, color="gold"),
                                 name="MLP"), row=1, col=1)
figPVCom2.append_trace(go.Scatter(y=predictionsVLSTM[:,0], x=datosPV_test.index,
                                 mode='lines', line=dict(width=2, color="lightgreen"),
                                 name="Vanilla LSTM"), row=1, col=1)
figPVCom2.append_trace(go.Scatter(y=predictionsLSTM[:,0], x=datosPV_test.index,
                                 mode='lines', line=dict(width=2, color="fuchsia"),
                                 name="LSTM"), row=1, col=1)
figPVCom2.append_trace(go.Scatter(y=predictionsCLSTM[:,0], x=datosPV_test.index,
                                 mode='lines', line=dict(width=2, color="gray"),
                                name="ConvLSTM"), row=1, col=1)


figPVCom2.append_trace(go.Scatter(y=datosPV_test.KWHD[95:,], x=datosPV_test.index,
                                 mode='lines', line=dict(width=2, color="blue"),
                                 name="Test data", showlegend=False), row=2, col=1)
figPVCom2.append_trace(go.Scatter(y=predictionsMLP2[:,0], x=datosPV_test.index,
                                 mode='lines', line=dict(width=2, color="gold"),
                                 name="MLP", showlegend=False), row=2, col=1)
figPVCom2.append_trace(go.Scatter(y=predictionsVLSTM2[:,0], x=datosPV_test.index,
                                 mode='lines', line=dict(width=2, color="lightgreen"),
                                 name="Vanilla LSTM", showlegend=False), row=2, col=1)
figPVCom2.append_trace(go.Scatter(y=predictionsLSTM2[:,0], x=datosPV_test.index,
                                 mode='lines', line=dict(width=2, color="fuchsia"),
                                 name="LSTM", showlegend=False), row=2, col=1)
figPVCom2.append_trace(go.Scatter(y=predictionsCLSTM2[:,0], x=datosPV_test.index,
                                 mode='lines', line=dict(width=2, color="gray"),
                                name="ConvLSTM", showlegend=False), row=2, col=1)


figPVCom2.append_trace(go.Scatter(y=datosPV_test.KWHD[95:,], x=datosPV_test.index,
                                 mode='lines', line=dict(width=2, color="blue"),
                                 name="Test data", showlegend=False), row=3, col=1)
figPVCom2.append_trace(go.Scatter(y=predictionsMLP3[:,0], x=datosPV_test.index,
                                 mode='lines', line=dict(width=2, color="gold"),
                                 name="MLP", showlegend=False), row=3, col=1)
figPVCom2.append_trace(go.Scatter(y=predictionsVLSTM3[:,0], x=datosPV_test.index,
                                 mode='lines', line=dict(width=2, color="lightgreen"),
                                 name="Vanilla LSTM", showlegend=False), row=3, col=1)
figPVCom2.append_trace(go.Scatter(y=predictionsLSTM3[:,0], x=datosPV_test.index,
                                 mode='lines', line=dict(width=2, color="fuchsia"),
                                 name="LSTM", showlegend=False), row=3, col=1)
figPVCom2.append_trace(go.Scatter(y=predictionsCLSTM3[:,0], x=datosPV_test.index,
                                 mode='lines', line=dict(width=2, color="gray"),
                                name="ConvLSTM", showlegend=False), row=3, col=1)



figPVCom2.update_layout(title='',
                      paper_bgcolor='rgba(0,0,0,0)',
                      plot_bgcolor='rgba(0,0,0,0)',
                      width=1500,height=2400,
                      yaxis=dict(color="black"),
                      xaxis=dict(color="black"),
                      font=dict(family="Fira Sans, Condensed", color='black'),
                      legend=dict(orientation="h",yanchor="bottom",y=1.02,xanchor="right",x=1, font_size=25, font_color='black'),
                      showlegend=True)
figPVCom2.update_xaxes(range=['2016-07-17 00:00:00','2016-07-19 00:00:00'])    
figPVCom2.update_xaxes(showline=True, linewidth=1, linecolor='black', gridcolor='lightgray', mirror=False, title_text='<b>Time (15 min)</b>', titlefont_size=35, tickfont_size=28)
figPVCom2.update_yaxes(showline=True, linewidth=1, gridcolor='lightgray', linecolor='black', mirror=False, title_text='<b>PV Power Output (kWh)</b>', titlefont_size=35, tickfont_size=28)
figPVCom2.update_annotations(font=dict(size=35))
#figPVCom2.show()